In [1]:
from pdfminer.high_level import extract_text

text = extract_text("data/de/cyberport_20220730_Rechnung.pdf")
print(text)

-

Cyberpoft Store Leipzig, Neumarkt 29-33, 04109 Leipzig

Cyberport Store leاp2اg  Kassenkunde
Neumarkt  29-33
04109  Leipzig

II

Rechnung

Ihre  Referenznr.

0904483033  /  30-07-2022

Kunde

7163789

Rechnungsnr./-datum

6001215715  /  30.07.2022

Lieferscheinnr./-datum

605330978

/  30.07.2022

Bestelinr./-datum

904483033

/  30.07.2022

Verkaufsbelegart

Auftrag  Store

Zahlart

Kreditkarte  Mastercard  Store

T ransaktionsnummer

Seriennummer  TSE

19654627559810188739

30.07.2022  14:21:48  Uhr

Vorgangsbeginn

Vorgangsende

Seite

٦
Lieferscheindatum  entspricht  dem  tiefer-  bzw.  Leistungsdatum!

30.07.2022  14:21:50  Uhr
٦

؛

Pos.

Produkt-  bzw.  Leistungsbeschreibung

Menge  USt.  (%)

Einzelpreis

1

2

5C44-04G
Eve  Energy  -  2xSmarte  Steckdose  mit  Ver
5C44-٥49
Eve  Energy  -  Smarte  Steckdose  mit  Verbra

(  RV08L1A13251,  RV0811A13252  )

1

19,0  ٠/٠

42,90

19,0  %

Gesamtbetrag

Im  Gesamtbetrag  entfialten  sind:

Nettobetrag

Umsatzsteuer

EUR
19,00./. 

## Microsoft Markitdown

In [72]:
from markitdown import MarkItDown

md = MarkItDown(enable_plugins=False) # Set to True to enable plugins
result_engl = md.convert("data/digital/2512.04834v1.pdf")
content_engl = result_engl.text_content
result_ger = md.convert("data/digital/1708.02100v1.pdf")
content_ger = result_ger.text_content

Cannot set gray stroke color because /'P5' is an invalid float value
Cannot set gray non-stroke color because /'P5' is an invalid float value
Cannot set gray stroke color because /'P6' is an invalid float value
Cannot set gray non-stroke color because /'P6' is an invalid float value
Cannot set gray stroke color because /'P7' is an invalid float value
Cannot set gray non-stroke color because /'P7' is an invalid float value
Cannot set gray stroke color because /'P9' is an invalid float value
Cannot set gray non-stroke color because /'P9' is an invalid float value
Cannot set gray stroke color because /'P10' is an invalid float value
Cannot set gray non-stroke color because /'P10' is an invalid float value
Cannot set gray stroke color because /'P14' is an invalid float value
Cannot set gray non-stroke color because /'P14' is an invalid float value
Cannot set gray stroke color because /'P15' is an invalid float value
Cannot set gray non-stroke color because /'P15' is an invalid float value


## Stanza NLP preprocessing

### stanza setup

In [ ]:
from stanza.models.common.doc import Document
from stanza.pipeline.core import Pipeline
import stanza
import pprint


# -----------------------------
# 1. Download necessary models
# -----------------------------
stanza.download('multilingual')  # langid (multilingual)
stanza.download('en')  # English NLP models
stanza.download('de')  # German NLP models

# -----------------------------
# 2. Initialize pipelines
# -----------------------------
# Language identification pipeline
langid_pipeline = Pipeline(lang="multilingual", processors="langid")

# English NLP pipeline
nlp_en = Pipeline(lang="en", processors="tokenize,pos,lemma,ner")

# German NLP pipeline
nlp_de = Pipeline(lang="de", processors="tokenize,pos,lemma,ner")

# English transformer based pipeline 
transformer_nlp_en = Pipeline(lang="en", processors="tokenize,coref,pos,lemma,ner")

# German transformer based pipeline 
transformer_nlp_de = Pipeline(lang="de", processors="tokenize,coref,pos,lemma,ner")

# Dictionary to select pipeline dynamically
pipelines = {
    'en': nlp_en,
    'de': nlp_de,
    'en_t': transformer_nlp_en,
    'de_t': transformer_nlp_de
}

### NLP Pipeline & sample run

In [106]:
# -----------------------------
# 3. Function to process texts
# -----------------------------
def process_texts(texts, model=None):
    """
    Takes a list of texts, detects language, and processes each text
    with the appropriate NLP pipeline (English/German).
    Returns a list of dicts with tokens, POS, lemmas, NER, language, confidence.
    """
    # Wrap texts into Document objects for langid
    docs = [Document([], text=text) for text in texts]

    # Run language identification
    langid_pipeline(docs)

    processed_docs = []
    results = []
    
    for num, doc in enumerate(docs):
        lang = doc.lang
        # Choose the correct pipeline; default to English
        if lang == "en":
            nlp = pipelines.get(lang, nlp_en)
        elif lang == "de":
            nlp = pipelines.get(lang, nlp_de)
        elif (lang == "en") & (model == "t"):
            nlp = pipelines.get("en_t", transformer_nlp_en)
        elif (lang == "de") & (model == "t"):
            nlp = pipelines.get("de_t", transformer_nlp_de)            
        else: # Fallback
            nlp = pipelines.get(lang, nlp_en)

        # Process the text
        processed_doc = nlp(doc.text)

        processed_docs.append(processed_doc)

        results.append(processed_doc.to_dict())


    return processed_docs, results

# -----------------------------
# 4. Example usage
# -----------------------------
texts = []

texts.append(result_engl.text_content)

texts.append(result_ger.text_content)

processed_docs, results = process_texts(texts, "t")

#print("{:C}".format(processed_docs[0][0]))

#pprint.pprint(processed_docs[0])

In [97]:
from stanza.utils.conll import CoNLL

conlls = []

for i in range(len(results)):
    conll = CoNLL.convert_dict(results[i])
    conlls.append(conll)


In [101]:
processed_docs[1].entities[:5]

[{
   "text": "Maximilian Eibl",
   "type": "PER",
   "start_char": 72,
   "end_char": 87
 },
 {
   "text": "Martin Gaedke",
   "type": "PER",
   "start_char": 89,
   "end_char": 102
 },
 {
   "text": "INFORMATIK 2017,\nLecture Notes in Informatics (LNI), Gesellschaft für Informatik",
   "type": "MISC",
   "start_char": 112,
   "end_char": 192
 },
 {
   "text": "Bonn",
   "type": "LOC",
   "start_char": 194,
   "end_char": 198
 },
 {
   "text": "Andreas Arzt1",
   "type": "PER",
   "start_char": 268,
   "end_char": 281
 }]

In [88]:
for i in range(len(processed_docs)):
    print(len(processed_docs[i].entities))

1102
318


In [57]:
nlp_en.processors["ner"].get_known_tags()

['CARDINAL',
 'DATE',
 'EVENT',
 'FAC',
 'GPE',
 'LANGUAGE',
 'LAW',
 'LOC',
 'MONEY',
 'NORP',
 'ORDINAL',
 'ORG',
 'PERCENT',
 'PERSON',
 'PRODUCT',
 'QUANTITY',
 'TIME',
 'WORK_OF_ART']

In [58]:
nlp_de.processors["ner"].get_known_tags()

['LOC', 'MISC', 'ORG', 'PER']